In [2]:
# Importation des bibliothèques nécessaires
import torch
import torchvision
import torchvision.transforms as transforms
import ssl
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import time
# Résoudre les problèmes SSL pour télécharger les datasets dans Google Colab
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Transformation des images : Conversion en tensor et normalisation
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir l'image en tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalisation des images en niveaux de gris
])

# Téléchargement du jeu de données MNIST (train et test)
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.10MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 65.3kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:06<00:00, 244kB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 12.2MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
import torch.nn as nn
import torch.optim as optim

# Définir l'architecture du CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:
# Définir le périphérique (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de l'appareil : {device}")


Utilisation de l'appareil : cuda


In [6]:
# Créer le modèle et l'envoyer sur le GPU si disponible
model = CNN().to(device)

In [7]:
# Fonction de perte et optimiseur
criterion = nn.CrossEntropyLoss()  # Perte pour classification multi-classes
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimiseur Adam


In [9]:
# Nombre d'époques pour l'entraînement
import numpy as np
num_epochs = 10
# Variables pour calculer les métriques
all_labels = []
all_preds = []

# Enregistrer le temps d'entraînement total
start_time = time.time()
for epoch in range(num_epochs):
    model.train()  # Passer le modèle en mode entraînement
    running_loss = 0.0
    epoch_labels = []
    epoch_preds = []

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Déplacer les données sur le GPU si disponible

        optimizer.zero_grad()  # Réinitialiser les gradients
        outputs = model(inputs)  # Faire une prédiction
        loss = criterion(outputs, labels)  # Calculer la perte
        loss.backward()  # Propagation arrière
        optimizer.step()  # Mettre à jour les poids

        running_loss += loss.item()  # Ajouter la perte à la somme des pertes

        # Sauvegarder les prédictions et les labels pour le calcul de l'accuracy et du F1 score
        _, predicted = torch.max(outputs, 1)
        epoch_labels.extend(labels.cpu().numpy())
        epoch_preds.extend(predicted.cpu().numpy())

    # Calcul des métriques pour cette époque
    accuracy = 100 * (sum(np.array(epoch_labels) == np.array(epoch_preds)) / len(epoch_labels))
    f1 = f1_score(epoch_labels, epoch_preds, average='weighted')

    print(f"Époque [{epoch+1}/{num_epochs}], Perte: {running_loss / len(trainloader):.4f}, "
          f"Accuracy: {accuracy:.2f}%, F1 Score: {f1:.2f}")

# Enregistrer le temps total d'entraînement
training_time = time.time() - start_time
print(f"\nTemps d'entraînement total : {training_time / 60:.2f} minutes")


Époque [1/10], Perte: 0.0460, Accuracy: 98.61%, F1 Score: 0.99
Époque [2/10], Perte: 0.0336, Accuracy: 98.95%, F1 Score: 0.99
Époque [3/10], Perte: 0.0265, Accuracy: 99.14%, F1 Score: 0.99
Époque [4/10], Perte: 0.0218, Accuracy: 99.34%, F1 Score: 0.99
Époque [5/10], Perte: 0.0201, Accuracy: 99.41%, F1 Score: 0.99
Époque [6/10], Perte: 0.0165, Accuracy: 99.51%, F1 Score: 1.00
Époque [7/10], Perte: 0.0152, Accuracy: 99.56%, F1 Score: 1.00
Époque [8/10], Perte: 0.0170, Accuracy: 99.56%, F1 Score: 1.00
Époque [9/10], Perte: 0.0134, Accuracy: 99.62%, F1 Score: 1.00
Époque [10/10], Perte: 0.0147, Accuracy: 99.63%, F1 Score: 1.00

Temps d'entraînement total : 12.61 minutes


In [10]:
# Tester le modèle sur le jeu de données de test
model.eval()  # Passer le modèle en mode évaluation
correct = 0
total = 0
with torch.no_grad():  # Ne pas calculer les gradients pendant l'évaluation
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Précision du modèle sur les données de test : {100 * correct / total:.2f}%")

Précision du modèle sur les données de test : 99.02%
